In [151]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np
from sklearn.linear_model import LinearRegression
from plotly.subplots import make_subplots

df_neuschnee = pd.read_csv('Neuschnee.csv')

In [170]:
def sk_linreg(df, coef=False):
    df.dropna(inplace=True)
    X = np.array(df['Year']).reshape(-1, 1)
    reg = LinearRegression().fit(X, df.iloc[:,-1])
    df['Linear Regression Line'] = reg.predict(X.reshape(-1, 1))
    return df, reg.coef_ if coef else df

def plot(df_):
    data_name = df_.columns[-1]
    fig = go.Figure()
    for region in np.unique(df_['Region']):
        df = df_[df_['Region']==region]
        fig.add_trace(go.Scatter(x=df['Year'], y=df[data_name], name=region))

    fig.update_layout(title=data_name + ' pro Jahr',
                   xaxis_title='Jahr',
                   yaxis_title=data_name)
    return fig

def plot_linreg(df, data_name='Neuschnee'):
    df.dropna(inplace=True)
    df, coef = sk_linreg(df, True)
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df['Year'], y=df[data_name], name=data_name))
    fig.add_trace(go.Scatter(x=df['Year'], y=df['Linear Regression Line'], name='Linear Regression'))
    fig.update_layout(title='{} pro Jahr. Veränderung von {:.2} pro Jahr'.format(data_name, coef[0]),
    xaxis_title='Jahr',
    yaxis_title=data_name)
    return fig

def plot_linreg_subplots(df, data_name='Neuschnee'):
    coefs = []
    for region in np.unique(df['Region']):
        df_ = df[df['Region']==region].copy()
        df_, coef = sk_linreg(df_, True)
        coefs.append(coef[0])
    
    subplots_titles = []
    for coef, region in zip(coefs, np.unique(df['Region'])):
        subplots_titles.append('{} in {}. Veränderung von {:.2} pro Jahr.'.format(data_name, region, coef))
    
    nr_region = len(np.unique(df['Region']))
    fig = make_subplots(rows=nr_region, cols=1, subplot_titles=subplots_titles)
    
    for i, region in enumerate(np.unique(df['Region']), 1):

        df_ = df[df['Region']==region].copy()
        df_, coef = sk_linreg(df_, True)
        fig.append_trace(go.Scatter(x=df_['Year'], y=df_[data_name],
                                    name=data_name),
                         row = i, col = 1)
        fig.append_trace(go.Scatter(x=df_['Year'], y=df_['Linear Regression Line'], name=np.round(coef[0],2)),
                         row = i, col = 1)
    
    
    fig.update_layout(height=3000, width=600, title_text=data_name+ " und Lineare Regression.",
                      showlegend=False, xaxis_title='Jahr', yaxis_title=data_name)
    return fig

In [140]:
fig = plot(df_neuschnee)
fig.show()

In [141]:
df_ns_mean = df_neuschnee.groupby('Year', as_index=False).mean()
plot_linreg(df_ns_mean).show()

In [156]:
df_ns_mean_1965 = df_ns_mean[df_ns_mean['Year']>=1965].copy()
plot_linreg(df_ns_mean_1965).show()


In [171]:
fig = plot_linreg_subplots(df_neuschnee)
fig.show()


In [172]:
df_jahresniederschlag = pd.read_csv('Jahresniederschlag.csv')
fig = plot(df_jahresniederschlag)
fig.show()

In [145]:
df_regen_mean = df_jahresniederschlag.groupby('Year', as_index=False).mean()
plot_linreg(df_regen_mean, 'Jahresniederschlag').show()

In [174]:
fig = plot_linreg_subplots(df_jahresniederschlag, 'Jahresniederschlag')
fig.show()